# YouTube Comments Scrapping 
    
   **BY - Mohit Kundu**

In [36]:
#importing the required Libraries
import pandas as pd
import numpy as np
from selenium import webdriver 
import pathlib
import time
from datetime import datetime

In [37]:
# installing the required libraries 

#!pip install selenium
#!apt-get update 
#!apt install chromium-chromedriver

In [39]:
# chromedriver should be installed and it's path may vary depending where it is installed
# Specifying the path accordingly
locationOfWebdriver = "C:/Users/in/Documents/WhatsAppBotTut/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(locationOfWebdriver , options=chrome_options)

search_item = ['Books', 'Man', 'boy', 'girls', 'smart phones', 'Data Science', 'Machine Learning', 'Calculator', 'Laptops', 'Python', 'Anaconda', 'Tiger', 'Lions', 'Home', 'Engineer']

Making a function for getting the links from the web

In [40]:
# Function to get links
def get_links(item) :    
    query = urllib.parse.quote(item)
    url = "https://www.youtube.com/results?search_query=" + query

    chrome_option = Options()
    chrome_option.add_argument("--incognito")
    driver = webdriver.Chrome(executable_path = locationOfWebdriver, options = chrome_option)


    driver.get(url)
    driver.execute_script("window.scrollTo(0, 1000000)")

    driver.maximize_window()
    container=driver.find_elements_by_xpath('//*[(@id = "video-title")]')

    links=[]
    for page in container:
        url=page.get_attribute("href")
        links.append(url)

    return list(set(links))

In [33]:
# Getting the links

video_links = []
for item in search_item :
    video_links = video_links + get_links(item)

In [22]:
#video_links   #to see the output remove '#'

In [23]:
#Converting the links into dataframe and then saving the links 
df = pd.DataFrame(video_links, columns = ['Links'])
#df
df.to_csv('Links.csv') 

In [41]:
# Reading the saved links
links_df = pd.read_csv("Links.csv", index_col = 0)
links_df[links_df['Links'].isnull()].count

<bound method DataFrame.count of     Links
69    NaN
137   NaN
215   NaN
252   NaN
333   NaN
373   NaN
520   NaN>

In [42]:
# Dropping the NAN values from the data
df = links_df.dropna()
print(len(df))
# Converting into array and taking 500 links
videos = np.array(df['Links'][0:500])
print(len(videos))
# initialising the dictionary to store the extracted data from the links
videos_dictionary = {}
videos_dictionary['Comments'] = {}
videos_dictionary['Video Link'] = {}


557
500


Defining the function to scrap the required information from the links

In [43]:
def scrap(url,i):
    
    print('Fetched date and time - ',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    
    try :
        videos_dictionary['Video Link'].update({i : url})
    except:        
         videos_dictionary['Video Link'].update({i :''})
    
    driver.get(url)
        
    time.sleep(3)

    comments =  driver.find_element_by_xpath('//*[@id="comments"]')
    driver.execute_script("arguments[0].scrollIntoView();", comments)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load".
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far.
        time.sleep(1)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
    comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    
    comment_string = ''

    for user, comment in zip(username_elems, comment_elems):
        comment_string = comment_string + user.text + ' - ' + comment.text + '\n'
    try :
        videos_dictionary['Comments'].update({i : comment_string})
    except :
        videos_dictionary['Comments'].update({i : ''})

First check the file exist to save the data that has been scrapped from the links. If the file does not exist, then create one. If it exist then check whether it is empty or not.

If not empty, then extract data drom the links not present in the file.

In [48]:
# Making a file if not present and also checking the file 
start = 0
name = 'Youtube_scrapping_comments.csv'
file_name = pathlib.Path(name)
if file_name.exists ():
    print ("File exist.")
    try :
        print("Reading the file now")
        df_temp = pd.read_csv(name, index_col = 0)

        start = (len(df_temp) )
    except:
        print("File is empty")
        
    
else:
    print ("File does not exist\n", "Creating the file")
    file = open(name,"w+")
    print("File created successfully with filename - ",name)
    
print("Starting from position - ", start)

File exist.
Reading the file now
Starting from position -  279


In [49]:
# Scrapping the comments
for num in range (start, len(videos)):
    url = videos[num]
    print("Loop entered")
    try :
        print("getting link-",num)
        driver.get(url)
    except :
        print("Not getting")
        continue

    print("=" * 40)  # Shows in terminal when a new airline is being scraped
    print("Scraping " + url)
    
    scrap(url,num)
    print("=" * 40)  # Shows in terminal when a new airline is being scraped   
    
print("Process ended success")

Loop entered
getting link- 279
Scraping https://www.youtube.com/watch?v=_KomtGXzT3o
Fetched date and time -  25/07/2020 18:44:46
Loop entered
getting link- 280
Scraping https://www.youtube.com/watch?v=AXPnz3iebkM
Fetched date and time -  25/07/2020 18:44:57
Loop entered
getting link- 281
Scraping https://www.youtube.com/watch?v=6_0-uEgEcI0
Fetched date and time -  25/07/2020 18:45:13
Loop entered
getting link- 282
Scraping https://www.youtube.com/watch?v=SYPUiDvObVA
Fetched date and time -  25/07/2020 18:45:22
Loop entered
getting link- 283
Scraping https://www.youtube.com/watch?v=xW5xMJ7T3QM
Fetched date and time -  25/07/2020 18:45:40
Loop entered
getting link- 284
Scraping https://www.youtube.com/watch?v=ndxUg-sG2BM
Fetched date and time -  25/07/2020 18:46:49
Loop entered
getting link- 285
Scraping https://www.youtube.com/watch?v=FJliMTWEOfs
Fetched date and time -  25/07/2020 18:47:28
Loop entered
getting link- 286
Scraping https://www.youtube.com/watch?v=PAc5ToNhGNw
Fetched date 

Scraping https://www.youtube.com/watch?v=3xX997E42Ro
Fetched date and time -  25/07/2020 19:12:08
Loop entered
getting link- 319
Scraping https://www.youtube.com/watch?v=vg-96Got_PA
Fetched date and time -  25/07/2020 19:13:04
Loop entered
getting link- 320
Scraping https://www.youtube.com/watch?v=k_GM1JA608Y
Fetched date and time -  25/07/2020 19:13:36
Loop entered
getting link- 321
Scraping https://www.youtube.com/watch?v=_jHeXEJseBk
Fetched date and time -  25/07/2020 19:14:03
Loop entered
getting link- 322
Scraping https://www.youtube.com/watch?v=yxZQPcMTdJw
Fetched date and time -  25/07/2020 19:14:14
Loop entered
getting link- 323
Scraping https://www.youtube.com/watch?v=JKAuJma-ulY
Fetched date and time -  25/07/2020 19:16:00
Loop entered
getting link- 324
Scraping https://www.youtube.com/watch?v=MljYZi1IcWw
Fetched date and time -  25/07/2020 19:18:37
Loop entered
getting link- 325
Scraping https://www.youtube.com/watch?v=dxPhJ0wfp0g
Fetched date and time -  25/07/2020 19:18:52

Scraping https://www.youtube.com/watch?v=k9TUPpGqYTo
Fetched date and time -  25/07/2020 19:44:45
Loop entered
getting link- 358
Scraping https://www.youtube.com/watch?v=76vTbQ5Wf-A
Fetched date and time -  25/07/2020 19:45:13
Loop entered
getting link- 359
Scraping https://www.youtube.com/watch?v=YYXdXT2l-Gg
Fetched date and time -  25/07/2020 19:45:32
Loop entered
getting link- 360
Scraping https://www.youtube.com/watch?v=Eijmv78piU4
Fetched date and time -  25/07/2020 19:46:50
Loop entered
getting link- 361
Scraping https://www.youtube.com/watch?v=XGf2GcyHPhc
Fetched date and time -  25/07/2020 19:47:00
Loop entered
getting link- 362
Scraping https://www.youtube.com/watch?v=kLZuut1fYzQ
Fetched date and time -  25/07/2020 19:48:19
Loop entered
getting link- 363
Scraping https://www.youtube.com/watch?v=PAPG8NI0iUI
Fetched date and time -  25/07/2020 19:49:53
Loop entered
getting link- 364
Scraping https://www.youtube.com/watch?v=A32HF0SsEzI
Fetched date and time -  25/07/2020 19:50:27

Scraping https://www.youtube.com/watch?v=sF9doyJbFLk
Fetched date and time -  25/07/2020 20:13:02
Loop entered
getting link- 397
Scraping https://www.youtube.com/watch?v=59P4JmAexnc
Fetched date and time -  25/07/2020 20:14:38
Loop entered
getting link- 398
Scraping https://www.youtube.com/watch?v=x8Z0PcwOI_c
Fetched date and time -  25/07/2020 20:15:50
Loop entered
getting link- 399
Scraping https://www.youtube.com/watch?v=Rm-x4maMAks
Fetched date and time -  25/07/2020 20:16:24
Loop entered
getting link- 400
Scraping https://www.youtube.com/watch?v=SUKDey0-7bs
Fetched date and time -  25/07/2020 20:16:47
Loop entered
getting link- 401
Scraping https://www.youtube.com/watch?v=mVPoHpazkVM
Fetched date and time -  25/07/2020 20:17:25
Loop entered
getting link- 402
Scraping https://www.youtube.com/watch?v=Q7bzLQlXBj4
Fetched date and time -  25/07/2020 20:17:36
Loop entered
getting link- 403
Scraping https://www.youtube.com/watch?v=Snz0VAwJXY4
Fetched date and time -  25/07/2020 20:17:51

Scraping https://www.youtube.com/watch?v=_UbDeqPdUek
Fetched date and time -  25/07/2020 20:45:17
Loop entered
getting link- 436
Scraping https://www.youtube.com/watch?v=RXHKxmgBd7s
Fetched date and time -  25/07/2020 20:46:34
Loop entered
getting link- 437
Scraping https://www.youtube.com/watch?v=xmU0s2QtaEY
Fetched date and time -  25/07/2020 20:46:41
Loop entered
getting link- 438
Scraping https://www.youtube.com/watch?v=rMs06PfdJlU
Fetched date and time -  25/07/2020 20:47:25
Loop entered
getting link- 439
Scraping https://www.youtube.com/watch?v=xeqJU7PLseg
Fetched date and time -  25/07/2020 20:48:39
Loop entered
getting link- 440
Scraping https://www.youtube.com/watch?v=SAcpESN_Fk4
Fetched date and time -  25/07/2020 20:49:52
Loop entered
getting link- 441
Scraping https://www.youtube.com/watch?v=-Xw21JYIXjM
Fetched date and time -  25/07/2020 20:51:26
Loop entered
getting link- 442
Scraping https://www.youtube.com/watch?v=eqxTdSybKww
Fetched date and time -  25/07/2020 20:51:34

Scraping https://www.youtube.com/watch?v=rMs06PfdJlU
Fetched date and time -  25/07/2020 21:10:09
Loop entered
getting link- 475
Scraping https://www.youtube.com/watch?v=-olqqcTBbAI
Fetched date and time -  25/07/2020 21:12:11
Loop entered
getting link- 476
Scraping https://www.youtube.com/watch?v=4BPhSCEeymc
Fetched date and time -  25/07/2020 21:12:51
Loop entered
getting link- 477
Scraping https://www.youtube.com/watch?v=sPyAQQklc1s
Fetched date and time -  25/07/2020 21:12:59
Loop entered
getting link- 478
Scraping https://www.youtube.com/watch?v=GibiNy4d4gc
Fetched date and time -  25/07/2020 21:13:32
Loop entered
getting link- 479
Scraping https://www.youtube.com/watch?v=a5V6gdu5ih8
Fetched date and time -  25/07/2020 21:13:39
Loop entered
getting link- 480
Scraping https://www.youtube.com/watch?v=dJU9ZkPorDE
Fetched date and time -  25/07/2020 21:15:38
Loop entered
getting link- 481
Scraping https://www.youtube.com/watch?v=lQHwkN_VQQM
Fetched date and time -  25/07/2020 21:15:48

In [50]:
# Converting into table for better visuals
data = pd.DataFrame.from_dict(videos_dictionary)
try:
    result = pd.concat([df_temp,data], ignore_index = True)
    result
except :
    result = data
result

,Comments,Video Link
0,"Yoo Se Na - ""You know what sexy?....\nBooks.""\...",https://www.youtube.com/watch?v=bWWTk-pZX5g
1,"- Hello Aluxers, what's your favorite book se...",https://www.youtube.com/watch?v=gyK9USvrvDQ
2,Brian Perkins - I love audiobooks. I use youtu...,https://www.youtube.com/watch?v=GihybX7JyG4
3,Ishaan Vohra - Watching a smart guy do smart t...,https://www.youtube.com/watch?v=4f2OCYaqLdE
4,Sabine's Book Nook - Us both having a book buy...,https://www.youtube.com/watch?v=imA5NPX4ucU
...,...,...
514,Its Fawn - I wish you safety from Covid-19❤\nS...,https://www.youtube.com/watch?v=dcHWmDQYzyU
515,Séries & Letras - so perfect\nKatsudon monster...,https://www.youtube.com/watch?v=lbSOLBMUvIE
516,,https://www.youtube.com/watch?v=ZmF8_g5H8fg
517,- BRUTAL! \n\nGuetta The Best ^^ \n\nMucha fu...,https://www.youtube.com/watch?v=Vr2FLgmWCJA


In [51]:
# Dropping NAN values
results = result.dropna()
# writing to the file (remove "#" to write to the file)
#results.to_csv(name)
results

,Comments,Video Link
0,"Yoo Se Na - ""You know what sexy?....\nBooks.""\...",https://www.youtube.com/watch?v=bWWTk-pZX5g
1,"- Hello Aluxers, what's your favorite book se...",https://www.youtube.com/watch?v=gyK9USvrvDQ
2,Brian Perkins - I love audiobooks. I use youtu...,https://www.youtube.com/watch?v=GihybX7JyG4
3,Ishaan Vohra - Watching a smart guy do smart t...,https://www.youtube.com/watch?v=4f2OCYaqLdE
4,Sabine's Book Nook - Us both having a book buy...,https://www.youtube.com/watch?v=imA5NPX4ucU
...,...,...
513,"XcuteR - Panda's daily routine -\nEating, bath...",https://www.youtube.com/watch?v=q-SdZe-cEFk
514,Its Fawn - I wish you safety from Covid-19❤\nS...,https://www.youtube.com/watch?v=dcHWmDQYzyU
515,Séries & Letras - so perfect\nKatsudon monster...,https://www.youtube.com/watch?v=lbSOLBMUvIE
516,,https://www.youtube.com/watch?v=ZmF8_g5H8fg


In [52]:
# Emptying the dictionary
videos_dictionary = {}

In [53]:
#Closing the driver 
driver.close()

# END

**Made By - Mohit Kundu**